https://chatgpt.com/share/687d132f-70f0-8003-8800-16c45376f179

i extract subtitle from youtube video
the video is 2hours long
i want summarize with Key points of the video using llm model
so i chunk subtitle every 10 minutes and input each chunk to model to summarize
eg. 1 time summarize for 1 chunk
and then I combine all the sumarize together
I use this method because llm model seem to cunfuse when I input the large amount of data into it
so I chunk it first and do all above
is it a good method or any method suggest that better than this? 

to do
- try chunking in token 
    - 1,000–2,000 tokens per chunk is the safe sweet spot
    - Add 10–20% overlap to catch mid-topic cuts

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.youtube.com/watch?v=ZCoK9nakeJE"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.title.string.replace(" - YouTube", "").strip()

In [2]:
import re

# get video title name
def clean_title(text):
    return re.sub(r'[^0-9a-zA-Z\u0E00-\u0E7F\.]', '', text)
title = clean_title(text=title)

# get video id
match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
if match:
    video_id = match.group(1)

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi

# Fetch transcript (auto-captions or uploaded)
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['th', 'en'])

# Optionally, save to file
file_name = f'{title}_{video_id}'
with open(f"{file_name}_subtitle.txt", "w", encoding="utf-8") as f:
    for entry in transcript:
        f.write(f"{entry['start']:.2f}s: {entry['text']}\n")
        # f.write(f"{entry['text']}\n")

In [4]:
import re

def load_subtitles(file_path):
    subtitles = []
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            match = re.match(r'([0-9.]+)s:\s(.+)', line.strip())
            if match:
                start_time = float(match.group(1))
                text = match.group(2)
                subtitles.append({'start': start_time, 'text': text})
    return subtitles

def chunk_subtitles(subtitles, chunk_size=60, overlap=20):
    """
    Chunk subtitles into segments of `chunk_size` seconds with `overlap` seconds.
    """
    chunks = []
    max_time = subtitles[-1]['start']
    start_time = 0

    while start_time <= max_time:
        end_time = start_time + chunk_size
        chunk_text = []
        for entry in subtitles:
            if start_time <= entry['start'] < end_time:
                chunk_text.append(entry['text'])
        if chunk_text:
            chunks.append({
                'start': start_time,
                'end': end_time,
                'text': ' '.join(chunk_text)
            })
        start_time += chunk_size - overlap
    return chunks

subtitles = load_subtitles(f"{file_name}_subtitle.txt")
chunk_dict = chunk_subtitles(subtitles, chunk_size=600, overlap=60)
# chunk_dict = chunk_subtitles(subtitles, chunk_size=300, overlap=30)

In [ ]:
import re
from pythainlp.tokenize import word_tokenize

def read_text_file(file_path):
    """Read text from a file and return as a string."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def remove_timestamps(text):
    """Remove timestamps like '0.12s:' at the start of each line."""
    cleaned_text = re.sub(r'^\s*\d+\.\d+s:\s*', '', text, flags=re.MULTILINE)
    return cleaned_text

def remove_newlines(text):
    """Remove newline characters."""
    return text.replace('\n', '')

def chunk_thai_text(text, max_tokens, overlap):
    """
    Chunk Thai text using PyThaiNLP word tokenizer with overlap.
    """
    words = word_tokenize(text, keep_whitespace=False)
    chunks = []
    start = 0

    while start < len(words):
        end = min(start + max_tokens, len(words))
        chunk = ''.join(words[start:end])  # Join with no spaces for Thai
        chunks.append(chunk)
        start += max_tokens - overlap

    return chunks

# Example usage:
file_path = f"{file_name}_subtitle.txt"
text = read_text_file(file_path)
text = remove_timestamps(text)
text = remove_newlines(text)
chunks = chunk_thai_text(text, max_tokens=1000, overlap=200)


In [6]:
# # subtitle = [subtitle['text'] for subtitle in subtitles]
# with open(f"{file_name}_subtitle.txt", 'r') as f:
#     contents = f.read()

In [7]:
title = file_name.split('_')[0]

## Use “Key points extraction” instead of general summary
## Use hierarchical summarization

In [ ]:
# === PROMPTS ===
KEY_POINTS_PROMPT = """
You are a helpful assistant.
Below is a section of a video transcript.
Extract the key points using clear bullet points.
Label each point with one of these: [Fact], [Idea], [Quote], [Question], [Action].
Be precise. Keep points short and factual.
Do not add commentary or extra text.

Transcript:
\"\"\"
{chunk_text}
\"\"\"
"""

SECTION_MERGE_PROMPT = """
Below are key points extracted from multiple chunks of a section.
Merge them into a single list:
- Remove duplicates or near-duplicates.
- Group similar ideas if needed.
- Keep clear bullet points.
- Preserve labels like [Fact], [Idea], etc.

Key Points:
\"\"\"
{key_points_text}
\"\"\"
"""

FINAL_SUMMARY_PROMPT = """
Below are the section-level key points for the entire video.
Write a single, well-organized summary:
- Merge and deduplicate.
- Organize by themes.
- Keep [Fact], [Idea], [Quote], [Action], etc.
- Present clearly for a slide deck or notes.
- Do not add fluff.

All Section Points:
\"\"\"
{all_sections_text}
\"\"\"
"""

# === HELPERS ===

import ollama
def call_llm(prompt: str):
    response = ollama.chat(
        model="llama3",
        messages=[
            {"role": "system", "content": "You are a precise simplify assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["message"]["content"]

In [11]:
# === PIPELINE ===

chunk_key_points = []
print("Extracting key points for each chunk...")
for i, chunk in enumerate(chunks):
    prompt = KEY_POINTS_PROMPT.format(chunk_text=chunk)
    result = call_llm(prompt)
    print(f"Chunk {i+1} done.")
    chunk_key_points.append(result)
    print(result)

# Group into sections, e.g., 3 chunks per section
SECTION_SIZE = 3
section_key_points = []

print("\nMerging chunks into sections...")
for i in range(0, len(chunk_key_points), SECTION_SIZE):
    group = chunk_key_points[i:i+SECTION_SIZE]
    group_text = "\n\n".join(group)
    prompt = SECTION_MERGE_PROMPT.format(key_points_text=group_text)
    result = call_llm(prompt)
    print(f"Section {i//SECTION_SIZE + 1} done.")
    section_key_points.append(result)

# Final merge
print("\nMerging all sections into final summary...")
all_sections_text = "\n\n".join(section_key_points)
final_prompt = FINAL_SUMMARY_PROMPT.format(all_sections_text=all_sections_text)
final_summary = call_llm(final_prompt)

print("\n✅ Final Summary:\n")
print(final_summary)

Extracting key points for each chunk...
Chunk 1 done.
Here are the key points from the transcript:

* [Fact]: The company discussed is DNAy Ceramic, also known as DCC.
* [Idea]: The speaker thinks that the company's value lies in its low price-to-earnings (PE) ratio and dividend yield.
* [Quote]: "คือในช่องผมนี่ก็จะมีทำเรื่องเกี่ยวกับหุ้นประมาณว่าแบบเป็นหุ้นValueหุ้นที่มีราคาค่อนข้างถูกถ้าเทียบกับPEหรือว่าค่าต่างๆนะครับ"
* [Action]: The speaker is planning to create a video about the company and its investment opportunities.
* [Fact]: The company DNAy Ceramic is a Thai ceramic products manufacturer.
* [Idea]: The speaker thinks that the company's growth potential lies in its ability to expand globally.
* [Quote]: "เพราะว่าทรงร้านน่ะมันไม่ใช่ร้านแบบแนวขายของแพงขายของพรีเมี่มติดแอร์อะไรประมาณเนี้ยนะครับ"
* [Action]: The speaker is encouraging viewers to follow the company's progress and consider investing in it.
* [Fact]: The company DNAy Ceramic has a dividend yield of around 5%.
* [Ide

In [10]:
with open(f"{file_name}_summary_v3.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

## Try Map-Reduce with embeddings (advanced)